In [1]:
import numpy as np
import astropy as apy

from tardis.io.atom_data import AtomData
from tardis.io.config_reader import Configuration

from tardis.model import Radial1DModel
from tardis.plasma.base import BasePlasma
from tardis.energy_input.indivisible_packets import main_gamma_ray_loop

import matplotlib.pyplot as plt

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

#### Configuration Object

In [2]:
config = Configuration.from_yaml("../../../tardis_example.yml")

#### Model Object

In [3]:
# Create model
model = Radial1DModel.from_config(config)

#### AtomData Object

In [4]:
# Create AtomData object
config.atom_data = "kurucz_cd23_chianti_H_He.h5"
atomic_data = AtomData.from_hdf(config.atom_data)

#### Plasma Object

In [5]:
from tardis.plasma.properties import Density, Abundance, IsotopeAbundance, IsotopeNumberDensity, AtomicData, AtomicMass, IsotopeMass, NumberDensity, SelectedAtoms

# plasma properties input
input = [Density, Abundance, IsotopeAbundance, AtomicData, AtomicMass, IsotopeNumberDensity, NumberDensity, SelectedAtoms, IsotopeMass]

In [6]:
#Create BasePlasma Object
plasma = BasePlasma(
        plasma_properties=input,
        density = model.density,
        abundance = model.abundance,
        isotope_abundance = model.raw_isotope_abundance,
        atomic_data = atomic_data
    )
    

AttributeError: 'NoneType' object has no attribute 'to_numpy'

The error above arises because **isotope_properties** in ```PlasmaPropertyCollection``` has no data for calculation, as there is no isotope-nuclide in **config.model.abundances**.

In [7]:
config.model.abundances

{'type': 'uniform',
 'O': 0.19,
 'Mg': 0.03,
 'Si': 0.52,
 'S': 0.19,
 'Ar': 0.04,
 'Ca': 0.03}

This can be be verified with the **raw_isotope_abundance** data in the model object, which comes out to be empty.

In [8]:
model.raw_isotope_abundance

,,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
atomic_number,mass_number,,,,,,,,,,,,,,,,,,,,


### Using TARDIS

We thus take inspiration from the TARDIS infrastructure and use ```assemble_plasma()``` to make our plasma object even when there's no isotope available.

In [9]:
from tardis.plasma.standard_plasmas import assemble_plasma

plasma = assemble_plasma(
                config, model, atom_data = atomic_data
            )

/home/sherryff/MyProjects/tardis/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)
/home/sherryff/MyProjects/tardis/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning: invalid value encountered in true_divide
  (g_lower * n_upper) / (g_upper * n_lower)


In [10]:
#Setting up number of decays requested
num_decays = config.montecarlo.no_of_packets

In [12]:
(
    energy_df,
    energy_plot_df,
    escape_energy,
    decayed_packet_count,
    energy_plot_positrons,
    estimated_deposition
) = main_gamma_ray_loop(
    num_decays,
    model,
    plasma
)

AttributeError: 'BasePlasma' object has no attribute 'isotope_number_density'

Evidently, the ***Gamma-Ray-Transport Code*** ALWAYS requires data on isotopes in the model.